# Practical 2: Calculating gravity disturbances from absolute gravity

In practical 1, we learned how to process the raw gravimeter relative readings into absolute gravity values. These data are affected by the entire mass of the Earth, which masks the small variations associated with crustal geological structures (our targets). So we have to **remove the effect of all other masses** before we can interpret the data in terms of crustal geology.

We'll do this in the following steps:

1. Elimintate the effect of most of the Earth's mass and rotation by removing **normal gravity**
2. Remove the effect of topography above the ellipsoid by removing the gravity of a **Bouguer plate**
3. Isolate shallower and local crustal structures by removing a **regional field**

The **residual field** that is left after these steps should be close to the gravitational attraction resulting from our geological structures. Of course, the way we calculate all of them relies on approximations and more accurate methods exist but would be more than we can cover in a single practical.

## Goals

* Implement the normal gravity and Bouguer plate calculations in Python
* Calculate a residual gravity disturbance around Cape Town
* Correlate the observed residual field to known local geology

## Data 

For this practical, you'll need:

* The absolute gravity values for the survey around Cape Town. This should have been produced as an output of practical 1. You can also download a CSV file with calculated values: [`cape-town-survey-absolute.csv`](https://raw.githubusercontent.com/leouieda/gravity-processing/main/data/cape-town-survey-absolute.csv) 

Place the file into the **same folder as this notebook**. Your should folder structure now should look something like this:

```
gravity-practicals/
├── cape-town-survey-absolute.csv
├── practical2.ipynb
├── cape-town-surveys/
│   ├── cape-town-gravity-day-1.csv
│   ├── cape-town-gravity-day-2.csv
│   ├── cape-town-gravity-day-3.csv
│   ├── cape-town-gravity-day-4.csv
│   └── cape-town-gravity-day-5.csv
├── gravimeter-scaling.csv
└── practical1.ipynb
```

The first few lines of the data file:

```
longitude,latitude,easting,northing,elevation,gravity
18.34444,-34.12971,255105.43,6220276.33,32.2,979656.12
18.37418,-34.19583,258037.64,6213013.01,18.4,979666.46
18.40388,-34.23972,260899.25,6208214.71,25.0,979671.03
18.50333,-34.03555,269506.57,6231090.44,15.1,979640.22
```

The data are a subset of the public domain data for Southern Africa distributed by the [NOAA National Centers for Environmental Information](https://www.ngdc.noaa.gov/).

## Survey location

The surveys were caried out near Cape Town, South Africa. Most points fall around False Bay and the Cape Peninsula, stretching a bit northwards. See the Wikipedia page on the [Geology of Cape Town](https://en.wikipedia.org/wiki/Geology_of_Cape_Town) for details on the geologic context.

In [ ]:
from IPython.display import IFrame

In [ ]:
IFrame(src="https://www.google.com/maps/d/embed?mid=1FunoJz1_shZ3wxfNFpg5FH33qkOg3tie", width=800, height=600)

## Import the required libraries

To deal with this type of tabular data, all we need are numpy and matplotlib. We'll also be using [scikit-learn](https://scikit-learn.org) to calculate the regional field.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Both of these will be used to remove the regional trend
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

## Read and plot the data

First thing we need to do is read in and plot the absolute gravity data. We'll do this with numpy using the same arguments we used previously. While we're at it, we'll also place each column into a separate variable to make them easier to use.

## Remove normal gravity

### What is normal gravity?

The gravity measured at the stations includes:

* The attraction of all masses of the Earth (absolutely everything)
* The centrifugal acceleration due to the Earth's rotation

To isolate the effect of local geology, we have to remove as much of this signal as we can. The first step is to approximate the Earth for a **rotating oblate ellipsoid**:

<img src="https://upload.wikimedia.org/wikipedia/commons/9/98/Ellipsoid_revolution_oblate_aac.svg" style="width: 500px; margin: 10px auto;">
<p style="text-align: center;">
Figure: Oblate ellipsoid with semimajor axis $a$, semiminor axis $c$, and origin at $O$. Credit: <a href="https://commons.wikimedia.org/wiki/File:Ellipsoid_revolution_oblate_aac.svg">Peter Mercator</a> (CC-BY-SA)    
</p>

The ellipsoid should have:

1. The same mass as the Earth
1. Rotate at the same angular velocity
1. Have the approximate dimensions of the Earth

If we can calculate the gravity of such an ellipsoid and remove it from our data, we can remove the vast majority of the signal, as well as the centrifugal acceleration (since the ellipsoid is also rotating). The gravity acceleration (gravitational + centrifugal) of a reference ellipsoid is called **normal gravity** and is often represented as $\gamma$.

We can calculate normal gravity **on the surface of the ellipsoid** using the [Somigliana equation](https://en.wikipedia.org/wiki/Theoretical_gravity#Somigliana_Formula):

$$
\gamma_0 (\theta) = \dfrac{a\ \gamma_a \cos^2 \theta + c\ \gamma_c \sin^2 \theta}{\sqrt{a^2 \cos^2 \theta + c^2 \sin^2 \theta}}
$$

in which $\theta$ is the latitude, $a$ and $c$ are the semimajor and seminor axis, and $\gamma_a$ and $\gamma_c$ are the normal gravity at the equator and the poles, respectively. It may seem strange that we need to know normal gravity at the pole and equator in order to calculate normal gravity elsewhere but these values can be calculated from the properties of the ellipsoid (axis, mass, and angular velocity).

For this practical, we will use the [WGS84 reference ellipsoid](https://en.wikipedia.org/wiki/World_Geodetic_System):

| | WGS84 |
|:----|:----|
| semimajor axis $a$ | 6378137 m |
| semiminor axis $c$ | 6356752 m |
| gravity at the equator $\gamma_a$ | 978032.53359 mGal |
| gravity at the poles $\gamma_c$ | 983218.49379 mGal |

### Normal gravity changes with latitude

Notice that the Somigliana equation only depends on the latitude of the computation point. That is because the calculation is done on the surface of the ellipsoid (zero height) and the field is symmetric with longitude. To visualize this, let's plot normal gravity for all latitudes.

### Normal gravity changes with height

The Somigliana equation doesn't take heights into account since this simplification is what allows the equation to be simple and compact. However, **our data are not on the surface of the ellipsoid** so we can't just remove the value given by the equation. We need to add a height component to normal gravity so that we can remove the effect of the ellipsoidal Earth from our data.

We know that the **derivative of normal gravity with height** at the surface of the ellipsoid is approximately -0.3086 mGal/m (known as the **free-air correction**). We can use this to adjust our normal gravity calculation:

$$
\gamma(\theta, h) \approx \gamma_0(\theta) - 0.3086 h
$$

Here, $h$ is the **height above the ellipsoid** of the computation point.

**Note**: There are equations that calculate normal gravity at any point analytically (e.g., [Li & Götze (2001)](https://doi.org/10.1190/1.1487109)) so **the free-air correction is no longer necessary**. The equation is more complicated, which is why we don't use it here. See [Boule](https://www.fatiando.org/boule/latest/) for a Python library that implements these equations.

---

## Your turn

Calculate normal gravity $\gamma(\theta, h)$ for the Cape Town survey points and make a pseudo-color map of the results. Then, remove normal gravity from the observed gravity and make a pseudo-color map of the results (observed gravity - normal gravity). Then think about and discuss the following points: 

* How does normal gravity vary along the survey?
* How close is normal gravity to representing the actual measured gravity (is it off by much)?
* What accounts for the signal in the map of observed - normal gravity (what is left after we remove normal gravity)?

---

## Removing topography

When we remove normal gravity from observed gravity, we are calculating a quantity know as the **gravity disturbance**:

$$
\delta g (P) = g (P) - \gamma (P)
$$

The disturbance is due largely to the difference in the density distribution of the real Earth and the Normal Earth (ellipsoid). For example:

* The real Earth has topography above and below the surface of the ellipsoid
* The real Earth has lateral changes in density
* The layers of the real Earth have variable thickness

What we are after as geoscientists are the last 2 item. But in order to see them, we need to remove the gravitational effect of topography (which we can assume we know to a good degree of accuracy). A simplified way of calculating the gravitational effect of topography is by using the Bouguer plate approximation:

$$
g_{\text{topo}} \approx 2 \pi G \rho D
$$

in which $G$ is the gravitational constant and $D$ is the thickness of the plate. In our case, the plate thickness would be the height of topography at a particular point. 

**Note**: See [this (unpublished) paper](https://www.leouieda.com/pdf/use-the-disturbance.pdf) and the references therein for more details on gravity disturbances versus gravity anomalies.

---

## Removing the regional field

The Bouguer (topography-free) gravity disturbance is largely dominated by effect of the Moho. In continental lithosphere, the Moho is deeper than the reference (the Moho of the Normal Earth) which generates a negative signal. In the oceans, it is the opposite effect. This trend needs to be removed from our data if we wish to see the smaller-scale crustal effects.

One way of removing this **regional field** is by approximating it by a low degree 2D polynomial (2nd or 3rd degree, for example). The process is as follows:

1. Fit a 2D polynomial of a given degree to the observations
1. Calculate the polynomial values on the observation points (this is the regional field)
1. Subtract the regional field from the observations to arrive at the **residual field**

We can do the polynomial fitting with the [scikit-learn library](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html). We'll use this here because it already comes pre-installed with Anaconda. However, there is a better alternative implemented in the [Verde library](https://www.fatiando.org/verde/v1.5.0/gallery/trend.html#sphx-glr-gallery-trend-py) which can be [installed on your computer](https://www.fatiando.org/verde/v1.5.0/install.html#installing-with-conda) if you wish.

---

## Your turn

Calculate the residual field and make a pseudo-color map of it with an appropriate colormap. Compare and correlate the signal you see in your map with the known [geology of Cape Town](https://en.wikipedia.org/wiki/Geology_of_Cape_Town). Discuss your thoughts and findings with your classmates (self-organize for this). While discussing, think about the following points:

* Can known geology explain the patter of positives and negatives observed in the data? 
* Why does/doesn't it and how would you justify your answer?
* How does the residual field change when you change the degree of the regional polynomial? 
* What would be your advice to others for choosing a polynomial degree?
* What simplifications went into producing this residual field (from the raw data until now)?
* How could these simplifications impact your interpretation of the data?